In [1]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy.dialects.postgresql import JSON, JSONB
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import json
import csv
import datetime
from dateutil.parser import parser
import os
from pprint import pprint
from pathlib import Path
import glob
#from odo import odo
import importlib
import itertools
import sklearn.cluster
import pandas as pd
import numpy as np
import math
import operator
import sklearn
import matplotlib.pyplot as plt

#importlib.invalidate_caches()
%load_ext autoreload
%autoreload 2

#### Reading in relevant dataframes

In [2]:
audio_features = pd.read_pickle('../Pickle_Files/audio_features.pkl')
audio_features = audio_features.drop(['time_signature','mode'],axis=1)

In [3]:
song_popularity = pd.read_pickle('../Pickle_Files/Song_Popularity.pkl')
song_popularity = song_popularity[['track_uri','Popularity_Rating_Normalized']]

In [4]:
clustered_songs = pd.read_pickle('../Pickle_Files/clustered_songs.pkl')
clustered_songs = clustered_songs.reset_index()

In [5]:
cluster = pd.read_pickle('../Pickle_Files/Cluster_Centers_PCA.pkl')

In [6]:
Compare = pd.read_pickle('../Pickle_Files/tracks.pkl')

#### Removes previously removed songs

In [7]:
Compare = pd.merge(audio_features[['track_uri']],Compare,on='track_uri')
audio_features = pd.merge(audio_features,Compare[['track_uri']],on='track_uri')
clustered_songs = pd.merge(clustered_songs,Compare[['track_uri']],on='track_uri')
song_popularity = pd.merge(song_popularity,Compare[['track_uri']],on='track_uri')

#### Merging dataframes for overall cosine ready dataframe

In [8]:
df1 = pd.merge(audio_features,song_popularity,on='track_uri')

In [9]:
df2 = pd.merge(df1,clustered_songs,on='track_uri')

In [10]:
cosine = pd.merge(df2,cluster,on='cluster')

In [11]:
cosine = cosine.drop(['cluster'],axis=1).set_index('track_uri')

#### Final Cosine Dataframe

In [12]:
cosine.head(5)

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence,Popularity_Rating_Normalized,principal component 1
track_uri,,,,,,,,,,,,
spotify:track:0XLOf9LhyazPX9Ld8jPiUq,0.0204,0.708,0.610,0.000004,2,0.0641,-4.522,0.0574,123.038,0.305,0.810959,-1.960397
spotify:track:5hfpu8saHLGedbHRtNCYjm,0.0202,0.961,0.625,0.000002,4,0.0895,-5.070,0.0686,124.992,0.223,0.747913,-1.960397
spotify:track:0CcQNd8CINkwQfe1RDtGV6,0.0524,0.779,0.787,0.000000,10,0.1400,-4.305,0.1080,124.982,0.708,0.909777,-1.960397
spotify:track:0gbBzIqrECJOEPvQJIBFs5,0.0162,0.616,0.714,0.000012,9,0.0817,-7.792,0.0982,128.924,0.240,0.832401,-1.960397
spotify:track:53DB6LJV9B8sz0p1s6tlGS,0.0524,0.523,0.805,0.000000,3,0.0914,-5.473,0.1920,125.358,0.602,0.774982,-1.960397


In [13]:
len(cosine)

2254670

In [14]:
cosine = cosine.sort_values('track_uri',ascending=True)

In [16]:
cosine.to_pickle('../Pickle_Files/cosine_dataframe.pkl')